face age gender emotion

In [15]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import model_from_json

# Load emotion model from JSON and weights
with open(r'C:\Users\Mongraal\PycharmProjects\JupyterProject\models\raf_model.json', 'r') as json_file:
    loaded_model_json = json_file.read()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights(r'C:\Users\Mongraal\PycharmProjects\JupyterProject\models\raf_model.weights.h5')
print("Emotion Model loaded successfully!")

# Load age and gender models
age_net = cv2.dnn.readNetFromCaffe(
    r'C:\Users\Mongraal\Downloads\Documents\age_deploy.prototxt',
    r'C:\Users\Mongraal\Downloads\Documents\age_net.caffemodel'
)

gender_net = cv2.dnn.readNetFromCaffe(
    r'C:\Users\Mongraal\Downloads\Documents\gender_deploy.prototxt',
    r'C:\Users\Mongraal\Downloads\Documents\gender_net.caffemodel'
)

print("Age and Gender Models loaded successfully.")

# Define lists for gender and age
gender_list = ['Male', 'Female']
age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']

# Define emotion labels
emotion_dict = {0: "Surprise", 1: "Disgust", 2: "Fear", 3: "Happiness", 4: "Sadness", 5: "Anger", 6: "Neutral"}

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()
mp_drawing = mp.solutions.drawing_utils

# Set desired resolution
desired_width = 1280
desired_height = 720

# Flags to toggle features
enable_gender = True
enable_age = True
enable_emotion = True

# Capture video from webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty frame.")
        continue

    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # Draw face mesh landmarks
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
            )

            # Extract face ROI
            height, width, _ = frame.shape
            x_min = int(min([lm.x for lm in face_landmarks.landmark]) * width)
            y_min = int(min([lm.y for lm in face_landmarks.landmark]) * height)
            x_max = int(max([lm.x for lm in face_landmarks.landmark]) * width)
            y_max = int(max([lm.y for lm in face_landmarks.landmark]) * height)
            face_roi = frame[y_min:y_max, x_min:x_max]

            if face_roi.size == 0:
                continue

            # Gender detection
            if enable_gender or enable_age:
                face_roi_age_gender = cv2.resize(face_roi, (227, 227))
                blob = cv2.dnn.blobFromImage(face_roi_age_gender, 1.0, (227, 227), (78.42633776, 87.76891434, 94.07864879), swapRB=False)

            if enable_gender:
                gender_net.setInput(blob)
                gender_preds = gender_net.forward()
                gender = gender_list[gender_preds[0].argmax()]

            if enable_age:
                age_net.setInput(blob)
                age_preds = age_net.forward()
                age = age_list[age_preds[0].argmax()]

            # Emotion detection
            if enable_emotion:
                cropped_img_emotion = cv2.resize(face_roi, (224, 224))
                cropped_img_emotion = np.expand_dims(cropped_img_emotion, 0) / 255.0
                emotion_prediction = emotion_model.predict(cropped_img_emotion)
                emotion_texts = [f'{emotion_dict[i]}: {emotion_prediction[0][i] * 100:.2f}%' for i in range(len(emotion_dict))]

            # Display results
            if enable_gender or enable_age:
                cv2.putText(frame, f'Gender: {gender if enable_gender else ""} Age: {age if enable_age else ""}',
                            (x_min, y_min - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            if enable_emotion:
                for i, text in enumerate(emotion_texts):
                    cv2.putText(frame, text, (x_min, y_min + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Face Analysis', frame)

    # Handle keyboard inputs for toggles
    key = cv2.waitKey(5) & 0xFF
    if key == ord('g'):  # Toggle gender detection
        enable_gender = not enable_gender
        print(f"Gender detection {'enabled' if enable_gender else 'disabled'}")
    elif key == ord('a'):  # Toggle age detection
        enable_age = not enable_age
        print(f"Age detection {'enabled' if enable_age else 'disabled'}")
    elif key == ord('e'):  # Toggle emotion detection
        enable_emotion = not enable_emotion
        print(f"Emotion detection {'enabled' if enable_emotion else 'disabled'}")
    elif key == ord('q'):  # Quit program
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Emotion Model loaded successfully!
Age and Gender Models loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━

In [5]:
pip install transformers torch tensorflow numpy pandas


SyntaxError: invalid syntax (382197014.py, line 1)

In [16]:
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Paths to your files
model_dir = r"C:\Users\Mongraal\Downloads\Documents\my_model"  # Directory containing model files
emotion_filter_path = r"C:\Users\Mongraal\Downloads\Documents\my_model\emotions_to_keep.json"
special_token_map_path = r"C:\Users\Mongraal\Downloads\Documents\my_model\special_tokens_map.json"

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_dir)

# Add special tokens to the tokenizer
with open(special_token_map_path, 'r') as file:
    special_tokens = json.load(file)
tokenizer.add_special_tokens({"additional_special_tokens": list(special_tokens.keys())})

# Load the model
model = BertForSequenceClassification.from_pretrained(model_dir)
model.resize_token_embeddings(len(tokenizer))

# Load the emotion filter
with open(emotion_filter_path, 'r') as file:
    emotion_to_keep = json.load(file)

# Function to predict emotion
def predict_emotion(text):
    # Replace special tokens in input text with their representations
    for token, token_id in special_tokens.items():
        text = text.replace(token, tokenizer.convert_ids_to_tokens(token_id))

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    emotion = emotion_to_keep.get(str(predicted_class), "Unknown Emotion")
    return emotion

# Test with an example
test_text = "[EMO_START] I am feeling very happy today! [EMO_END]"
predicted_emotion = predict_emotion(test_text)
print(f"Predicted Emotion: {predicted_emotion}")

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory C:\Users\Mongraal\Downloads\Documents\my_model.